# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [91]:
# import libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine

import nltk
# download libraries if necessary
# nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
# load data from database
engine = create_engine('sqlite:///CleanDatabase.db')
df = pd.read_sql_table('cleaned_data', engine)
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """ Function to process text in the following order: normalize, tokenize, 
        remove stop words, and lemmatize.
    Args:
        text (str): text to be processed
    Returns:
        tokens (list): list of tokens after processing text
    """

    # normalize text by removing punctuation and converting to lower case
    text = re.sub(r'[^\w\s]', ' ', text.lower())

    # tokenize text
    tokens = word_tokenize(text)

    # remove stop words and lemmatize
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word)
              for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier

In [11]:
# build pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
from sklearn.model_selection import train_test_split

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001F6F40FF558>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [14]:
# convert y_pred to dataframe
y_pred = pd.DataFrame(y_pred)
y_pred.columns = y_test.columns

In [15]:
from sklearn.metrics import classification_report

for column in y_test.columns:
    print(f'Category: {column}')
    print(classification_report(y_test[column], y_pred[column]))
    print('---------------------------------------------------------')
    print('\n')

Category: related
              precision    recall  f1-score   support

           0       0.74      0.12      0.21      1873
           1       0.78      0.99      0.87      5992

    accuracy                           0.78      7865
   macro avg       0.76      0.55      0.54      7865
weighted avg       0.77      0.78      0.71      7865

---------------------------------------------------------


Category: request
              precision    recall  f1-score   support

           0       0.84      1.00      0.91      6533
           1       0.78      0.08      0.14      1332

    accuracy                           0.84      7865
   macro avg       0.81      0.54      0.53      7865
weighted avg       0.83      0.84      0.78      7865

---------------------------------------------------------


Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00      0.00        36

    accura

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.91      1.00      0.95      7160
           1       0.88      0.03      0.06       705

    accuracy                           0.91      7865
   macro avg       0.90      0.52      0.51      7865
weighted avg       0.91      0.91      0.87      7865

---------------------------------------------------------


Category: clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7750
           1       1.00      0.03      0.07       115

    accuracy                           0.99      7865
   macro avg       0.99      0.52      0.53      7865
weighted avg       0.99      0.99      0.98      7865

---------------------------------------------------------


Category: money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7695
           1       0.50      0.01      0.01       170

    accuracy               

### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
from sklearn.model_selection import GridSearchCV

In [36]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'tfidf__sublinear_tf': (True, False),
    'clf__estimator__leaf_size': (20, 40),
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)
cv.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] clf__estimator__leaf_size=20, tfidf__sublinear_tf=True, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__leaf_size=20, tfidf__sublinear_tf=True, vect__ngram_range=(1, 1), score=0.229, total= 1.4min
[CV] clf__estimator__leaf_size=20, tfidf__sublinear_tf=True, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min remaining:    0.0s


[CV]  clf__estimator__leaf_size=20, tfidf__sublinear_tf=True, vect__ngram_range=(1, 1), score=0.204, total= 1.3min
[CV] clf__estimator__leaf_size=20, tfidf__sublinear_tf=True, vect__ngram_range=(1, 1) 


...

[CV]  clf__estimator__leaf_size=20, tfidf__sublinear_tf=True, vect__ngram_range=(1, 2), score=0.234, total= 1.4min
[CV] clf__estimator__leaf_size=20, tfidf__sublinear_tf=True, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 12.4min remaining:    0.0s


[CV]  clf__estimator__leaf_size=20, tfidf__sublinear_tf=True, vect__ngram_range=(1, 2), score=0.249, total= 1.4min
[CV] clf__estimator__leaf_size=20, tfidf__sublinear_tf=False, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__leaf_size=20, tfidf__sublinear_tf=False, vect__ngram_range=(1, 1), score=0.229, total= 1.3min
[CV] clf__estimator__leaf_size=20, tfidf__sublinear_tf=False, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__leaf_size=20, tfidf__sublinear_tf=False, vect__ngram_range=(1, 1), score=0.207, total= 1.3min
[CV] clf__estimator__leaf_size=20, tfidf__sublinear_tf=False, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__leaf_size=20, tfidf__sublinear_tf=False, vect__ngram_range=(1, 1), score=0.232, total= 1.4min
[CV] clf__estimator__leaf_size=20, tfidf__sublinear_tf=False, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__leaf_size=20, tfidf__sublinear_tf=False, vect__ngram_range=(1, 1), score=0.238, total= 1.4min
[CV] clf__estimator__leaf_size=20, tfidf__sublinear_tf=False, vect__n

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 75.9min finished


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001F6F40FF558>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=KNeighborsClassifier()))]),
             param_grid={'clf__estimator__leaf_size': (20, 40),
                         'tfidf__sublinear_tf': (True, False),
                         'vect__ngram_range': ((1, 1), (1, 2))},
             verbose=10)

In [39]:
cv.best_params_

{'clf__estimator__leaf_size': 20,
 'tfidf__sublinear_tf': False,
 'vect__ngram_range': (1, 1)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [143]:
# test function

def get_report(y_test, y_pred):
    
    results = []
    for column in y_test.columns:
        report = classification_report(y_test[column], y_pred[column], output_dict=True)
        results.append([report['accuracy'], report['macro avg']['f1-score'], report['weighted avg']['f1-score']])
        
    index = y_test.columns    
    columns = ['accuracy', 'f1-score macro avg', 'f1-score weighted avg']
    
    pd.DataFrame(results, index=index, columns=columns).mean()

In [37]:
# predict on test data
y_pred = cv.predict(X_test)

# convert y_pred to dataframe
y_pred = pd.DataFrame(y_pred)
y_pred.columns = y_test.columns

for column in y_test.columns:
    print(f'Category: {column}')
    print(classification_report(y_test[column], y_pred[column]))
    print('---------------------------------------------------------')
    print('\n')

Category: related
              precision    recall  f1-score   support

           0       0.74      0.12      0.21      1873
           1       0.78      0.99      0.87      5992

    accuracy                           0.78      7865
   macro avg       0.76      0.55      0.54      7865
weighted avg       0.77      0.78      0.71      7865

---------------------------------------------------------


Category: request
              precision    recall  f1-score   support

           0       0.84      1.00      0.91      6533
           1       0.78      0.08      0.14      1332

    accuracy                           0.84      7865
   macro avg       0.81      0.54      0.53      7865
weighted avg       0.83      0.84      0.78      7865

---------------------------------------------------------


Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00      0.00        36

    accura

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7750
           1       1.00      0.03      0.07       115

    accuracy                           0.99      7865
   macro avg       0.99      0.52      0.53      7865
weighted avg       0.99      0.99      0.98      7865

---------------------------------------------------------


Category: money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7695
           1       0.50      0.01      0.01       170

    accuracy                           0.98      7865
   macro avg       0.74      0.50      0.50      7865
weighted avg       0.97      0.98      0.97      7865

---------------------------------------------------------


Category: missing_people
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7773
           1       0.00      0.00      0.00        92

    accuracy         

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### Random Forest Classifier

In [61]:
from sklearn.ensemble import RandomForestClassifier

In [62]:
# build pipeline using RandomForestClassifier
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [47]:
# train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001F6F40FF558>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [48]:
# predict on test data
y_pred = pipeline.predict(X_test)

# convert y_pred to dataframe
y_pred = pd.DataFrame(y_pred)
y_pred.columns = y_test.columns

for column in y_test.columns:
    print(f'Category: {column}')
    print(classification_report(y_test[column], y_pred[column]))
    print('---------------------------------------------------------')
    print('\n')

Category: related
              precision    recall  f1-score   support

           0       0.71      0.39      0.51      1873
           1       0.83      0.95      0.89      5992

    accuracy                           0.82      7865
   macro avg       0.77      0.67      0.70      7865
weighted avg       0.80      0.82      0.80      7865

---------------------------------------------------------


Category: request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      6533
           1       0.83      0.50      0.63      1332

    accuracy                           0.90      7865
   macro avg       0.87      0.74      0.78      7865
weighted avg       0.89      0.90      0.89      7865

---------------------------------------------------------


Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00      0.00        36

    accura

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98      7620
           1       0.62      0.06      0.11       245

    accuracy                           0.97      7865
   macro avg       0.80      0.53      0.55      7865
weighted avg       0.96      0.97      0.96      7865

---------------------------------------------------------


Category: child_alone
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7865

    accuracy                           1.00      7865
   macro avg       1.00      1.00      1.00      7865
weighted avg       1.00      1.00      1.00      7865

---------------------------------------------------------


Category: water
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      7365
           1       0.88      0.36      0.51       500

    accuracy                           0.96      7865
   macro avg       0.92 

Category: other_weather
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      7450
           1       0.67      0.01      0.03       415

    accuracy                           0.95      7865
   macro avg       0.81      0.51      0.50      7865
weighted avg       0.93      0.95      0.92      7865

---------------------------------------------------------


Category: direct_report
              precision    recall  f1-score   support

           0       0.86      0.97      0.91      6321
           1       0.78      0.36      0.49      1544

    accuracy                           0.85      7865
   macro avg       0.82      0.67      0.70      7865
weighted avg       0.84      0.85      0.83      7865

---------------------------------------------------------




In [63]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'clf__estimator__bootstrap': (True, False),
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)
cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] clf__estimator__bootstrap=True, vect__ngram_range=(1, 1) ........


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__bootstrap=True, vect__ngram_range=(1, 1), score=0.273, total= 4.9min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.9min remaining:    0.0s


[CV] clf__estimator__bootstrap=True, vect__ngram_range=(1, 1) ........
[CV]  clf__estimator__bootstrap=True, vect__ngram_range=(1, 1), score=0.259, total= 6.8min


...

[CV] clf__estimator__bootstrap=True, vect__ngram_range=(1, 2) ........
[CV]  clf__estimator__bootstrap=True, vect__ngram_range=(1, 2), score=0.263, total=14.7min


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 104.8min remaining:    0.0s


[CV] clf__estimator__bootstrap=True, vect__ngram_range=(1, 2) ........
[CV]  clf__estimator__bootstrap=True, vect__ngram_range=(1, 2), score=0.276, total=16.3min
[CV] clf__estimator__bootstrap=False, vect__ngram_range=(1, 1) .......
[CV]  clf__estimator__bootstrap=False, vect__ngram_range=(1, 1), score=0.270, total= 7.5min
[CV] clf__estimator__bootstrap=False, vect__ngram_range=(1, 1) .......
[CV]  clf__estimator__bootstrap=False, vect__ngram_range=(1, 1), score=0.263, total= 7.6min
[CV] clf__estimator__bootstrap=False, vect__ngram_range=(1, 1) .......
[CV]  clf__estimator__bootstrap=False, vect__ngram_range=(1, 1), score=0.267, total= 7.6min
[CV] clf__estimator__bootstrap=False, vect__ngram_range=(1, 1) .......
[CV]  clf__estimator__bootstrap=False, vect__ngram_range=(1, 1), score=0.260, total= 7.9min
[CV] clf__estimator__bootstrap=False, vect__ngram_range=(1, 1) .......
[CV]  clf__estimator__bootstrap=False, vect__ngram_range=(1, 1), score=0.275, total= 8.0min
[CV] clf__estimator__bo

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 295.9min finished


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001F6F40FF558>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__bootstrap': (True, False),
                         'vect__ngram_range': ((1, 1), (1, 2))},
             verbose=10)

In [64]:
cv.best_params_

{'clf__estimator__bootstrap': False, 'vect__ngram_range': (1, 2)}

In [65]:
# predict on test data
y_pred = cv.predict(X_test)

# convert y_pred to dataframe
y_pred = pd.DataFrame(y_pred)
y_pred.columns = y_test.columns

for column in y_test.columns:
    print(f'Category: {column}')
    print(classification_report(y_test[column], y_pred[column]))
    print('---------------------------------------------------------')
    print('\n')

Category: related
              precision    recall  f1-score   support

           0       0.65      0.47      0.55      1873
           1       0.85      0.92      0.88      5992

    accuracy                           0.82      7865
   macro avg       0.75      0.70      0.72      7865
weighted avg       0.80      0.82      0.80      7865

---------------------------------------------------------


Category: request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      6533
           1       0.80      0.51      0.62      1332

    accuracy                           0.89      7865
   macro avg       0.85      0.74      0.78      7865
weighted avg       0.89      0.89      0.88      7865

---------------------------------------------------------


Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00      0.00        36

    accura

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7673
           1       0.78      0.13      0.22       192

    accuracy                           0.98      7865
   macro avg       0.88      0.56      0.61      7865
weighted avg       0.97      0.98      0.97      7865

---------------------------------------------------------


Category: security
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7721
           1       0.00      0.00      0.00       144

    accuracy                           0.98      7865
   macro avg       0.49      0.50      0.50      7865
weighted avg       0.96      0.98      0.97      7865

---------------------------------------------------------


Category: military
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      7620
           1       0.56      0.07      0.13       245

    accuracy            

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      7450
           1       0.44      0.03      0.06       415

    accuracy                           0.95      7865
   macro avg       0.69      0.52      0.52      7865
weighted avg       0.92      0.95      0.92      7865

---------------------------------------------------------


Category: direct_report
              precision    recall  f1-score   support

           0       0.86      0.97      0.91      6321
           1       0.74      0.36      0.48      1544

    accuracy                           0.85      7865
   macro avg       0.80      0.66      0.70      7865
weighted avg       0.84      0.85      0.83      7865

---------------------------------------------------------




#### Use custom transformers

In [73]:
from sklearn.base import BaseEstimator, TransformerMixin

In [118]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        lemmatizer = WordNetLemmatizer()
        for sentence in sentence_list:
            tokens = word_tokenize(sentence)
            clean_tokens = []
            for token in tokens:
                clean_tokens.append(
                    lemmatizer.lemmatize(token).lower().strip())
            pos_tags = nltk.pos_tag(clean_tokens)
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP']:
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [83]:
class TextLengthExtractor(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_len = pd.Series(X).apply(len)
        return pd.DataFrame(X_len)

In [86]:
from sklearn.pipeline import FeatureUnion

In [124]:
# build pipeline
pipeline = Pipeline([
    ('feature', FeatureUnion([
        ('nlp_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 2))),
            ('tfidf', TfidfTransformer())
        ])),
        ('sve', StartingVerbExtractor()),
        ('tle', TextLengthExtractor())
    ])
    ),
    ('clf', MultiOutputClassifier(RandomForestClassifier(bootstrap = False)))
])

In [125]:
# train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('feature',
                 FeatureUnion(transformer_list=[('nlp_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(ngram_range=(1,
                                                                                               2),
                                                                                  tokenizer=<function tokenize at 0x000001F6F40FF558>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('sve',
                                                 StartingVerbExtractor()),
                                                ('tle',
                                                 TextLengthExtractor())])),
                ('clf',
                 MultiOutputClassifier(estimato

In [126]:
# predict on test data
y_pred = pipeline.predict(X_test)

# convert y_pred to dataframe
y_pred = pd.DataFrame(y_pred)
y_pred.columns = y_test.columns

for column in y_test.columns:
    print(f'Category: {column}')
    print(classification_report(y_test[column], y_pred[column]))
    print('---------------------------------------------------------')
    print('\n')

Category: related
              precision    recall  f1-score   support

           0       0.69      0.40      0.51      1873
           1       0.83      0.94      0.89      5992

    accuracy                           0.81      7865
   macro avg       0.76      0.67      0.70      7865
weighted avg       0.80      0.81      0.80      7865

---------------------------------------------------------


Category: request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      6533
           1       0.83      0.51      0.63      1332

    accuracy                           0.90      7865
   macro avg       0.87      0.74      0.79      7865
weighted avg       0.89      0.90      0.89      7865

---------------------------------------------------------


Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00      0.00        36

    accura

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.92      0.99      0.96      7227
           1       0.57      0.08      0.14       638

    accuracy                           0.92      7865
   macro avg       0.75      0.54      0.55      7865
weighted avg       0.90      0.92      0.89      7865

---------------------------------------------------------


Category: medical_products
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      7447
           1       0.81      0.10      0.18       418

    accuracy                           0.95      7865
   macro avg       0.88      0.55      0.58      7865
weighted avg       0.94      0.95      0.93      7865

---------------------------------------------------------


Category: search_and_rescue
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7673
           1       0.73      0.12      0.21       192

    acc

              precision    recall  f1-score   support

           0       0.87      0.95      0.91      5702
           1       0.83      0.64      0.72      2163

    accuracy                           0.87      7865
   macro avg       0.85      0.79      0.82      7865
weighted avg       0.86      0.87      0.86      7865

---------------------------------------------------------


Category: floods
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      7242
           1       0.86      0.38      0.52       623

    accuracy                           0.95      7865
   macro avg       0.90      0.69      0.75      7865
weighted avg       0.94      0.95      0.94      7865

---------------------------------------------------------


Category: storm
              precision    recall  f1-score   support

           0       0.94      0.99      0.96      7127
           1       0.77      0.42      0.55       738

    accuracy                 

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.